## Q1. Chi Square Test

In [70]:
#TODO


import pandas as pd
import json
import numpy as np

business_path = "/Users/Andy/Desktop/yelp_academic_dataset_business.json"
business_doc=open(business_path, encoding="utf8").readlines()

busList = []
for bus in business_doc:
    busList.append(json.loads(bus))

dfBus = pd.DataFrame.from_records(busList)
dfBus = dfBus.dropna()
from pandas.io.json import json_normalize

In [71]:
open_df = dfBus.loc[:,'is_open']
estrella_df = dfBus.loc[:,'stars']

stars = estrella_df.values

bin_stars = [1 if (star > 2.5) else 0 for star in stars]

bin_stars = np.array(bin_stars) #1 for above 2.5 stars 0 for 2.5 and below

is_open = open_df.values #1 for open 0 for closed


##########################################################################
##########Data gathered, lets make observed & expected dataframes#########
##########################################################################

dfobserved =  pd.DataFrame(columns = ['Business Open' , 'Business Closed', 'Total Stars'], index=['Stars above 2.5', 'Stars below 2.5', 'Total Operating'])

##4 variables for each subcat
open_above = 0
open_below = 0
closed_above = 0
closed_below = 0

for idx, bus_status in enumerate(is_open):

    star_status = bin_stars[idx]

    if bus_status == 1:
        if star_status == 1:
            open_above += 1
        else:
            open_below += 1
    else:
        if star_status == 1:
            closed_above += 1
        else:
            closed_below += 1




In [72]:
#####lets fill in observed df########
dfobserved["Business Open"]["Stars above 2.5"]=open_above
dfobserved["Business Open"]["Stars below 2.5"]=open_below
dfobserved["Business Closed"]["Stars above 2.5"]=closed_above
dfobserved["Business Closed"]["Stars below 2.5"]=closed_below


#totals


dfobserved["Business Open"]["Total Operating"]=dfobserved[['Business Open']].dropna().sum()[0]
dfobserved["Business Closed"]["Total Operating"]=dfobserved[['Business Closed']].dropna().sum()[0]
dfobserved["Total Stars"]["Stars above 2.5"]=dfobserved.loc[["Stars above 2.5"]].T.dropna().sum()[0]
dfobserved["Total Stars"]["Stars below 2.5"]=dfobserved.loc[["Stars below 2.5"]].T.dropna().sum()[0]

dfobserved["Total Stars"]["Total Operating"]=dfobserved[["Total Stars"]].dropna().sum()[0]




Conclusions:	 your conclusions


In [135]:
import scipy.stats

dfNoTot=dfobserved.loc[["Stars above 2.5","Stars below 2.5"],['Business Open','Business Closed']]

contingency=pd.crosstab(bin_stars,is_open)
contingency.index=["Stars below 2.5","Stars above 2.5"]
contingency.columns=["Business Closed","Business Open"]

##contingecy and dfnoTot should be the same with some column//row swapping

chistat, pval, dof, expected = scipy.stats.chi2_contingency(contingency)


#TODO

conclusions = "The p value is %.3f%% for a 95%% confidence level, this is too low since it is below 5%%\n We reject the null hypothesis. A business's open/closed status and stars on yelp are not independent."%(pval*100)
print('Conclusions:\t', conclusions)

Conclusions:	 The p value is 2.605% for a 95% confidence level, this is too low since it is below 5%
 We reject the null hypothesis. A business's open/closed status and stars on yelp are not independent.


## Q2. Correlation coefficients

In [73]:
#TODO pearson coefficient


In [74]:
#TODO spearman coefficient


In [75]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q3. Association rule

In [76]:
import re
df_AZ = dfBus[dfBus["state"]=="AZ"]
raw_categories = df_AZ.loc[:,'categories']

cats = []
for cat in raw_categories:
    cats.append(cat)


#TODO


In [77]:
from apyori import apriori
nested = []

for desc in cats:
    sub = []
    key_words = re.findall('[a-zA-Z]+', desc)
    for word in key_words:
        sub.append(word)
    nested.append(sub)

rules = apriori(nested, min_support=0.03, min_confidence=0.7,min_lift=1,min_length=2)
list(rules)

[RelationRecord(items=frozenset({'Active', 'Fitness'}), support=0.032591791196159306, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Fitness'}), items_add=frozenset({'Active'}), confidence=0.9843430905377808, lift=16.986522714076127)]),
 RelationRecord(items=frozenset({'Active', 'Instruction'}), support=0.03256925192147317, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Instruction'}), items_add=frozenset({'Active'}), confidence=0.9958649207443143, lift=17.18535159030066)]),
 RelationRecord(items=frozenset({'Active', 'Life'}), support=0.05794847521806748, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Active'}), items_add=frozenset({'Life'}), confidence=1.0, lift=16.487179487179485), OrderedStatistic(items_base=frozenset({'Life'}), items_add=frozenset({'Active'}), confidence=0.955406911928651, lift=16.487179487179485)]),
 RelationRecord(items=frozenset({'Restaurants', 'American'}), support=0.05578470484819798, ordered_statistics=[OrderedStatistic(

In [78]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q4. Power law distribution

In [79]:
#TODO ccdf


In [80]:
#TODO powerlaw


In [81]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)


Conclusions:	 your conclusions
